# Google Gemini API

Play and learn.

### Dependencies

install dependencies first.



In [1]:
pip install -q -U google-generativeai

### API key
save the API key as `Gemini_key` in google environment variable.

In [4]:
# get the API key from google environement variable.
import os
from google.colab import userdata
import google.generativeai as genai


GOOGLE_API_KEY=userdata.get('Gemini_key')
genai.configure(api_key=GOOGLE_API_KEY)


## Markdown Dependencies
install required libraries for markdown and define a pretty markdown display function.


In [5]:
# load other necessary packages

import pathlib
import textwrap

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## GenAI available models:

These are all the available models in google generative AI package as of today:

In [6]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


## Generate text from text inputs

For text-only prompts, use the `gemini-pro` model.

write a message in prompt and pass it to gemini-pro model.

if interested you can look at other available reponse by running `response.candidates`.

In [7]:
#%%time
model    = genai.GenerativeModel('gemini-pro')
prompt   =  'write me a short poem about a red car'
response = model.generate_content(prompt)
to_markdown(response.text)

> In a world of colors, a vibrant hue,
> A fiery chariot, a scarlet view.
> Red car, you gleam, a passionate sight,
> Your sleek curves, a captivating flight.
> 
> Your glossy paint, a fiery kiss,
> Beneath the sun, your beauty's abyss.
> Heads turn, eyes dance, as you glide by,
> A symphony of hearts, you make them sigh.
> 
> Through city streets, you're a blazing trail,
> A rush of adrenaline, a thrilling tale.
> With every turn, you leave a mark,
> A scarlet streak, an indelible spark.
> 
> Red car, you're more than just a ride,
> You're a symbol of freedom, a source of pride.
> Your roar, a melody, your presence divine,
> In your embrace, dreams intertwine.
> 
> So let the wind flow, let the engine soar,
> Red car, you're a legend, forevermore.
> In your fiery dance, we find release,
> A symphony of speed, a moment of peace.

## Text/Image to text model:

Here we try to pass a prompt as well as pictures to `gemini-pro-vision` model and generate a response.

API is designed to handle multimodal prompts and returns a text output.

The following images are pulled from web and might not be available by the time you run the code. so feel free to change them as you wish.


In [10]:
# gettig some generic pictures from web
!curl -o image.jpg https://t0.gstatic.com/licensed-image?q=tbn:ANd9GcQ_Kevbk21QBRy-PgB4kQpS79brbmmEG7m3VOTShAn4PecDU5H5UxrJxE3Dw1JiaG17V88QIol19-3TM2wCHw
!curl -o image2.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Wikipedia-logo-v2-en.svg/800px-Wikipedia-logo-v2-en.svg.png
!curl -o cat.jpg https://upload.wikimedia.org/wikipedia/commons/1/15/Cat_August_2010-4.jpg
!curl -o dog.jpg https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Labrador_Retriever_portrait.jpg/1920px-Labrador_Retriever_portrait.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  405k  100  405k    0     0  5631k      0 --:--:-- --:--:-- --:--:-- 5705k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  217k  100  217k    0     0  2191k      0 --:--:-- --:--:-- --:--:-- 2199k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4523k  100 4523k    0     0  18.8M      0 --:--:-- --:--:-- --:--:-- 18.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  407k  100  407k    0     0  1877k      0 --:--:-- --:--:-- --:--:-- 1879k


Let's print those images and see if they are pulled from web!

In [13]:
import PIL.Image

img = PIL.Image.open('image2.jpg')


### Only passing an image to the model

if you only want to pass an image to the model here is how to do it.


In [14]:
model = genai.GenerativeModel('gemini-pro-vision')

response = model.generate_content(img)

to_markdown(response.text)

>  The Wikipedia logo is a puzzle sphere made up of puzzle pieces with different letters on each piece.

### Passing image & text to the model

The API handles multimodal prompts. here we pass in two image and ask a question about it

In [15]:
img_c = PIL.Image.open('cat.jpg')
img_d = PIL.Image.open('dog.jpg')

model = genai.GenerativeModel('gemini-pro-vision')
prompt='what is the difference between these images?'


response = model.generate_content([prompt, img_c,img_d], stream=True)
response.resolve()
to_markdown(response.text)

>  The first image is of a cat, and the second image is of a dog.

## Multi-turn conversation

you can set up the chat history when starting the model. Here we used no history. but you can chat a bit with the model and print out the chat history to get the structure and use that as an input for the model.

In [46]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

In [47]:
response = chat.send_message('I am 177 lb and 6 feet tall. what is my BMI?')
to_markdown(response.text)

> Your BMI (Body Mass Index) is 23.0.
> 
> BMI is calculated using the formula: BMI = weight (kg) / [height (m)]^2
> 
> To calculate your BMI in pounds and inches, you can use the following formula:
> 
> BMI = weight (lb) x 703 / [height (in)]^2
> 
> For your height of 6 feet, which is 72 inches, and a weight of 177 pounds, your BMI is:
> 
> BMI = 177 x 703 / (72 x 72) = 23.0
> 
> A BMI of 23.0 is considered to be in the normal range, according to the Centers for Disease Control and Prevention (CDC).

In [48]:
response = chat.send_message('I am hungry. recommend just one high protein meal')
to_markdown(response.text)

> **Grilled Salmon with Quinoa and Roasted Vegetables**
> 
> **Ingredients:**
> 
> * 1 salmon fillet (6-8 ounces)
> * 1 tablespoon olive oil
> * 1/4 teaspoon salt
> * 1/4 teaspoon black pepper
> * 1 cup quinoa, cooked
> * 1 cup roasted vegetables (such as broccoli, carrots, and zucchini)
> * 1/4 cup crumbled feta cheese (optional)
> * 1 tablespoon chopped fresh herbs (such as parsley or cilantro)
> 
> **Instructions:**
> 
> 1. Preheat oven to 400°F (200°C).
> 2. Season salmon fillet with salt and pepper.
> 3. Heat olive oil in a large skillet over medium heat.
> 4. Add salmon fillet to the skillet and cook for 3-4 minutes per side, or until cooked through.
> 5. While the salmon is cooking, roast the vegetables. Toss the vegetables with a little olive oil and salt and pepper. Spread the vegetables on a baking sheet and roast in the oven for 15-20 minutes, or until tender.
> 6. To assemble the meal, place the cooked salmon on a bed of quinoa. Top with roasted vegetables and crumbled feta cheese (if desired). Garnish with chopped fresh herbs.
> 
> This meal is high in protein, thanks to the salmon and quinoa. It is also a good source of healthy fats from the olive oil and feta cheese, and fiber from the quinoa and vegetables.
> 
> **Nutritional Information (per serving):**
> 
> * Calories: 450
> * Protein: 35 grams
> * Fat: 20 grams
> * Carbohydrates: 35 grams
> * Fiber: 5 grams

In [49]:
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

> **user**: I am 177 lb and 6 feet tall. what is my BMI?

> **model**: Your BMI (Body Mass Index) is 23.0.
> 
> BMI is calculated using the formula: BMI = weight (kg) / [height (m)]^2
> 
> To calculate your BMI in pounds and inches, you can use the following formula:
> 
> BMI = weight (lb) x 703 / [height (in)]^2
> 
> For your height of 6 feet, which is 72 inches, and a weight of 177 pounds, your BMI is:
> 
> BMI = 177 x 703 / (72 x 72) = 23.0
> 
> A BMI of 23.0 is considered to be in the normal range, according to the Centers for Disease Control and Prevention (CDC).

> **user**: I am hungry. recommend just one high protein meal

> **model**: **Grilled Salmon with Quinoa and Roasted Vegetables**
> 
> **Ingredients:**
> 
> * 1 salmon fillet (6-8 ounces)
> * 1 tablespoon olive oil
> * 1/4 teaspoon salt
> * 1/4 teaspoon black pepper
> * 1 cup quinoa, cooked
> * 1 cup roasted vegetables (such as broccoli, carrots, and zucchini)
> * 1/4 cup crumbled feta cheese (optional)
> * 1 tablespoon chopped fresh herbs (such as parsley or cilantro)
> 
> **Instructions:**
> 
> 1. Preheat oven to 400°F (200°C).
> 2. Season salmon fillet with salt and pepper.
> 3. Heat olive oil in a large skillet over medium heat.
> 4. Add salmon fillet to the skillet and cook for 3-4 minutes per side, or until cooked through.
> 5. While the salmon is cooking, roast the vegetables. Toss the vegetables with a little olive oil and salt and pepper. Spread the vegetables on a baking sheet and roast in the oven for 15-20 minutes, or until tender.
> 6. To assemble the meal, place the cooked salmon on a bed of quinoa. Top with roasted vegetables and crumbled feta cheese (if desired). Garnish with chopped fresh herbs.
> 
> This meal is high in protein, thanks to the salmon and quinoa. It is also a good source of healthy fats from the olive oil and feta cheese, and fiber from the quinoa and vegetables.
> 
> **Nutritional Information (per serving):**
> 
> * Calories: 450
> * Protein: 35 grams
> * Fat: 20 grams
> * Carbohydrates: 35 grams
> * Fiber: 5 grams